In [1]:
from sklearn.preprocessing import OneHotEncoder
from pymongo import MongoClient
from sets import Set
import numpy as np
import json

In [2]:
# prov = json.load(open('providers-OR.json'))
client = MongoClient('fc8iasm01', 27017)
prov_col = client.aca.prov
drug_col = client.aca.drug
plan_col = client.aca.plan

In [31]:
# get plan state - all combinations of drug_tier/pharmacy_type/copay_opt/coinsurance_opt
i=0
for p in plan_col.aggregate(
    [
        {'$match':{'plan_id':{'$in':['39424OR1240001','73836AK0750002','39424OR1480001']}}},     
        {'$unwind':'$formulary'},
        {'$unwind':'$formulary.cost_sharing'},        
        {'$group':{
                '_id':{
                    'plan':'$plan_id',
                    'tier':'$formulary.drug_tier', 
                    'pharmacy':'$formulary.cost_sharing.pharmacy_type',
                    'cp':'$formulary.cost_sharing.copay_opt',
                    'ci':'$formulary.cost_sharing.coinsurance_opt'                    
                    },
#                 'a_cp':{'$avg':'$formulary.cost_sharing.copay_amount'},
#                 'a_in':{'$avg':'$formulary.cost_sharing.coinsurance_rate'},
                'network':{'$first':'$network.network_tier'},
                'cnt':{'$sum':1},                
            }
        },
        {'$project':{
                '_id':0, 
                'plan':'$_id.plan',
                'tier':'$_id.tier',
                'pharmacy':'$_id.pharmacy',
                'copay_opt':'$_id.cp',
                'coinsurance_opt':'$_id.ci',
#                 'avg_copay':'$a_cp',
#                 'avg_ci_rate':'$a_in',
                'count':'$cnt',                
#                 'test':{'$sum':['$a_cp', '$a_in']},
                'network_tier':'$network'
            }
        
        },
#         {'$sort':{'plan':-1}}
    ]

):
    i+=1
    if i<5:
        print p
    
print i

{u'count': 1, u'copay_opt': u'NO-CHARGE', u'pharmacy': u'3-MONTH-IN-RETAIL', u'network_tier': [u'PREFERRED'], u'tier': u'BRAND', u'coinsurance_opt': None, u'plan': u'73836AK0750002'}
{u'count': 1, u'copay_opt': u'BEFORE-DEDUCTIBLE', u'pharmacy': u'3-MONTH-IN-MAIL', u'network_tier': [u'PREFERRED'], u'tier': u'VALUE', u'coinsurance_opt': None, u'plan': u'39424OR1240001'}
{u'count': 1, u'copay_opt': u'BEFORE-DEDUCTIBLE', u'pharmacy': u'1-MONTH-OUT-RETAIL', u'network_tier': [u'PREFERRED'], u'tier': u'SELECT', u'coinsurance_opt': None, u'plan': u'39424OR1240001'}
{u'count': 1, u'copay_opt': u'BEFORE-DEDUCTIBLE', u'pharmacy': u'1-MONTH-IN-RETAIL', u'network_tier': [u'PREFERRED'], u'tier': u'PREFERRED', u'coinsurance_opt': None, u'plan': u'39424OR1240001'}
47


In [35]:
# get the mean value of copay_amout and coinsurance_rate (over all tier/pharmacy/copay/coinsurance options) for a plan
i=0
for p in plan_col.aggregate(
    [
        {'$match':{'plan_id':{'$in':['39424OR1240001','73836AK0750002','39424OR1480001']}}},     
        {'$unwind':'$formulary'},
        {'$unwind':'$formulary.cost_sharing'},        
        {'$group':{
                '_id':{
                    'plan':'$plan_id',
#                     'tier':'$formulary.drug_tier', 
#                     'pharmacy':'$formulary.cost_sharing.pharmacy_type',
#                     'cp':'$formulary.cost_sharing.copay_opt',
#                     'ci':'$formulary.cost_sharing.coinsurance_opt'                    
                    },
                'a_cp':{'$avg':'$formulary.cost_sharing.copay_amount'},
                'a_in':{'$avg':'$formulary.cost_sharing.coinsurance_rate'},
                'network':{'$addToSet':'$network.network_tier'},
                'cnt':{'$sum':1},                
            }
        },
        {'$project':{
                '_id':0, 
                'plan':'$_id.plan',
#                 'tier':'$_id.tier',
#                 'pharmacy':'$_id.pharmacy',
#                 'copay_opt':'$_id.cp',
#                 'coinsurance_opt':'$_id.ci',
                'avg_copay':'$a_cp',
                'avg_ci_rate':'$a_in',
                'count':'$cnt',                
#                 'test':{'$sum':['$a_cp', '$a_in']},
                'network_tier':'$network'
            }
        
        },
        {'$sort':{'plan':1}}
    ]

):
    i+=1
    if i<5:
        print p
    
print i

{u'count': 16, u'avg_ci_rate': 0.10625000000000001, u'avg_copay': 38.125, u'plan': u'39424OR1240001', u'network_tier': [[u'PREFERRED']]}
{u'count': 16, u'avg_ci_rate': 0.24375, u'avg_copay': 0.25, u'plan': u'39424OR1480001', u'network_tier': [[u'PREFERRED']]}
{u'count': 15, u'avg_ci_rate': 0.14666666666666664, u'avg_copay': 12.0, u'plan': u'73836AK0750002', u'network_tier': [[u'PREFERRED']]}
3


In [161]:
plan_col.find_one()

{u'_id': ObjectId('5776d384b4ba2c8616d321b5'),
 u'formulary': [{u'cost_sharing': [{u'coinsurance_opt': None,
     u'coinsurance_rate': 0.0,
     u'copay_amount': 90.0,
     u'copay_opt': u'BEFORE-DEDUCTIBLE',
     u'pharmacy_type': u'1-MONTH-IN-RETAIL'},
    {u'coinsurance_opt': None,
     u'coinsurance_rate': 0.0,
     u'copay_amount': 30.0,
     u'copay_opt': u'BEFORE-DEDUCTIBLE',
     u'pharmacy_type': u'1-MONTH-OUT-RETAIL'},
    {u'coinsurance_opt': None,
     u'coinsurance_rate': 0.0,
     u'copay_amount': 270.0,
     u'copay_opt': u'BEFORE-DEDUCTIBLE',
     u'pharmacy_type': u'3-MONTH-IN-MAIL'}],
   u'drug_tier': u'PREFERRED',
   u'mail_order': True},
  {u'cost_sharing': [{u'coinsurance_opt': u'AFTER-DEDUCTIBLE',
     u'coinsurance_rate': 0.5,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE',
     u'pharmacy_type': u'1-MONTH-IN-RETAIL'}],
   u'drug_tier': u'SPECIALTY',
   u'mail_order': True},
  {u'cost_sharing': [{u'coinsurance_opt': None,
     u'coinsurance_rate': 0.0

In [105]:
for p in plan_col.find({'plan_id':'39424OR1240001'},{'_id':0,'formulary.cost_sharing.copay_amount':1}):
    print p

{u'formulary': [{u'cost_sharing': [{u'copay_amount': 90.0}, {u'copay_amount': 30.0}, {u'copay_amount': 270.0}]}, {u'cost_sharing': [{u'copay_amount': 0.0}]}, {u'cost_sharing': [{u'copay_amount': 10.0}, {u'copay_amount': 10.0}, {u'copay_amount': 90.0}]}, {u'cost_sharing': [{u'copay_amount': 10.0}, {u'copay_amount': 10.0}, {u'copay_amount': 90.0}]}, {u'cost_sharing': [{u'copay_amount': 0.0}, {u'copay_amount': 0.0}, {u'copay_amount': 0.0}]}, {u'cost_sharing': [{u'copay_amount': 0.0}]}, {u'cost_sharing': [{u'copay_amount': 0.0}, {u'copay_amount': 0.0}]}]}


In [36]:
# get state space of drugs - count for all combinations of drug_tier/step_therapy/quantity_limit/prior_authorization

i=0
for d in drug_col.aggregate(
    [           
        {'$unwind':'$plans'},
        {'$match':{'plans.plan_id':{'$in':['39424OR1240001','39424OR1480003']}}},     
        {'$group':{
                '_id':{'pl':'$plans.plan_id', 
                       'ti':'$plans.drug_tier', 
                       'st':'$plans.step_therapy',
                       'ql':'$plans.quantity_limit',
                       'pa':'$plans.prior_authorization',
                       }, 
                'cnt':{'$sum':1}
            }
        },
        {'$project':{
                'pid':'$_id.pl', 
                'tier':'$_id.ti', 
                'step_therapy':'$_id.st', 
                'quantity_limit':'$_id.ql', 
                'prior_authorization':'$_id.pa', 
                'count':'$cnt', '_id':0
            }
        },
        {'$sort':{'tier':1, 'pid':-1}}
    ]
):
    i+=1     
    if i<5:
        print d
        
print 'total:',i

{u'count': 85, u'quantity_limit': False, u'pid': u'39424OR1480003', u'step_therapy': False, u'prior_authorization': True, u'tier': u'BRAND'}
{u'count': 1722, u'quantity_limit': False, u'pid': u'39424OR1480003', u'step_therapy': False, u'prior_authorization': False, u'tier': u'BRAND'}
{u'count': 359, u'quantity_limit': True, u'pid': u'39424OR1480003', u'step_therapy': True, u'prior_authorization': False, u'tier': u'BRAND'}
{u'count': 115, u'quantity_limit': True, u'pid': u'39424OR1480003', u'step_therapy': False, u'prior_authorization': False, u'tier': u'BRAND'}
total: 66


In [34]:
# get rxnorm_id list for a group of plans
i=0
for d in drug_col.aggregate(
    [
        {'$unwind':'$plans'},
        {'$match':{'plans.plan_id':{'$in':['39424OR1240001']}}},        
        {'$group':{'_id':'$plans.plan_id', 'drugs': {'$addToSet':'$rxnorm_id'}}},
        {'$project':{'plan':'$_id', 'drug':'$drugs', '_id':0 }},
#         {'$project':{'plan':'$plans.plan_id', 'rx':'$rxnorm_id', '_id':0}},
#         {'$sort':{'rx':1}}
    ]
):
    i+=1
    if True:
        print d['plan'], len(d['drug'])
        
print i

39424OR1240001 8968
1


In [145]:
len( drug_col.find(
    {'plans.plan_id':'39424OR1320001'}, #'plans.drug_tier':{'$in':['SELECT', 'PREFERRED']}}, 
#     {'_id':0, 'rxnorm_id':1, 'plans':{ '$elemMatch':{'plan_id':'39424OR1320001'}}}
).distinct('rxnorm_id'))

# plan_id.count()

8968

In [129]:
plan_id.sort('plans.drug_tier')

In [152]:
plan_col.find_one({'plan_id':'39424OR1480001'})

{u'_id': ObjectId('5776d384b4ba2c8616d321bf'),
 u'formulary': [{u'cost_sharing': [{u'coinsurance_opt': u'AFTER-DEDUCTIBLE',
     u'coinsurance_rate': 0.4,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE',
     u'pharmacy_type': u'1-MONTH-IN-RETAIL'},
    {u'coinsurance_opt': u'AFTER-DEDUCTIBLE',
     u'coinsurance_rate': 0.4,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE',
     u'pharmacy_type': u'1-MONTH-OUT-RETAIL'},
    {u'coinsurance_opt': None,
     u'coinsurance_rate': 0.0,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE',
     u'pharmacy_type': u'3-MONTH-IN-MAIL'}],
   u'drug_tier': u'SELECT',
   u'mail_order': True},
  {u'cost_sharing': [{u'coinsurance_opt': u'AFTER-DEDUCTIBLE',
     u'coinsurance_rate': 0.4,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE',
     u'pharmacy_type': u'1-MONTH-IN-RETAIL'},
    {u'coinsurance_opt': u'AFTER-DEDUCTIBLE',
     u'coinsurance_rate': 0.4,
     u'copay_amount': 0.0,
     u'copay_opt': u'NO-CHARGE'

In [6]:
client.close()